<a href="https://colab.research.google.com/github/MOOwuttichai/BSC_DPDM2023/blob/main/LSTM_thai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 69.8 MB/s eta 0:00:00


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from pythainlp.tokenize import word_tokenize
import pandas as pd

In [11]:
df = pd.read_csv("/content/Data_model.csv")
labels = []
for sentiment in df['label']:
  if sentiment == "เล่าประสบการณ์ (tell experience)":
    labels.append(1) #เล่าประสบการณ์ (tell experience)
  elif sentiment == "คำถาม (Question)":
    labels.append(2) #ไม่มีประโยชน์/ไม่สำคัญ (useless/unimportant)
  else:
    labels.append(0)
df['label_num'] = labels

In [12]:
train_df_E, test_df_E = train_test_split(df, test_size=0.3, random_state=42) # 70/30 split, random_state for reproducibility
print("Training set size:", len(train_df_E))
print("Testing set size:", len(test_df_E))

Training set size: 2168
Testing set size: 930


In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from pythainlp.tokenize import word_tokenize

# โหลดข้อมูลจากไฟล์ CSV
data = pd.read_csv("/content/Data_model.csv")

# แยกข้อความและป้ายกำกับ
texts = data["comments"].astype(str).tolist()
labels = data["label"].astype(str).tolist()

# Tokenization โดยใช้ PyThaiNLP
tokenized_texts = [word_tokenize(text, keep_whitespace=False) for text in texts]

# แปลงคำเป็นตัวเลข
max_words = 5000
max_len = 20
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(tokenized_texts)
sequences = tokenizer.texts_to_sequences(tokenized_texts)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# แปลง labels เป็นตัวเลข
unique_labels = list(set(labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}
y = np.array([label_map[label] for label in labels])

# สร้างโมเดล LSTM
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(len(unique_labels), activation='softmax')
])

# คอมไพล์โมเดล
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# ฝึกโมเดล (ต้องมีข้อมูลเพิ่มเติมสำหรับการฝึกจริง)
model.fit(padded_sequences, y, epochs=10, batch_size=2)

# ฟังก์ชันสำหรับทำนายประเภทของข้อความ
def predict_category(text):
    tokenized = word_tokenize(text, keep_whitespace=False)
    sequence = tokenizer.texts_to_sequences([tokenized])
    padded = pad_sequences(sequence, maxlen=max_len, padding='post')
    pred = model.predict(padded)
    return unique_labels[np.argmax(pred)]

# ทดสอบการพยากรณ์
print(predict_category("เคยไปญี่ปุ่นมา สนุกมาก"))  # ควรได้ 'experience'
print(predict_category("มีใครรู้วิธีทำขนมครกไหม?"))  # ควรได้ 'question'


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1549/1549 ━━━━━━━━━━━━━━━━━━━━ 29s 16ms/step - accuracy: 0.9042 - loss: 0.2921
Epoch 2/10
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.9485 - loss: 0.1557
Epoch 3/10
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.9747 - loss: 0.0810
Epoch 4/10
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - accuracy: 0.9840 - loss: 0.0559
Epoch 5/10
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.9913 - loss: 0.0262
Epoch 6/10
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.9949 - loss: 0.0222
Epoch 7/10
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.9956 - loss: 0.0197
Epoch 8/10
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.9959 - loss: 0.0135
Epoch 9/10
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.9982 - loss: 0.0102
Epoch 10/10
1549/1549 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.9979 - loss: 0.0117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
ไม่มีประโยชน์/ไม่สำคัญ (useless/unimportant)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from pythainlp.tokenize import word_tokenize

# โหลดข้อมูลจากไฟล์ CSV
data = pd.read_csv("Data_model.csv")

# แยกข้อความและป้ายกำกับ
texts = data["comments"].astype(str).tolist()
labels = data["label"].astype(str).tolist()

# Tokenization โดยใช้ PyThaiNLP
tokenized_texts = [word_tokenize(text, keep_whitespace=False) for text in texts]

# แปลงคำเป็นตัวเลข
max_words = 5000
max_len = 20
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(tokenized_texts)
sequences = tokenizer.texts_to_sequences(tokenized_texts)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# แปลง labels เป็นตัวเลข
unique_labels = list(set(labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}
y = np.array([label_map[label] for label in labels])

# แบ่งข้อมูลเป็น train/test (70/30) โดยใช้ random_state=42
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.3, random_state=42)

# สร้างโมเดล LSTM
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(len(unique_labels), activation='softmax')
])

# คอมไพล์โมเดล
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# ฝึกโมเดล
model.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test))

# ฟังก์ชันสำหรับทำนายประเภทของข้อความ
def predict_category(text):
    tokenized = word_tokenize(text, keep_whitespace=False)
    sequence = tokenizer.texts_to_sequences([tokenized])
    padded = pad_sequences(sequence, maxlen=max_len, padding='post')
    pred = model.predict(padded)
    return unique_labels[np.argmax(pred)]

# ทดสอบการพยากรณ์
print(predict_category("เคยไปญี่ปุ่นมา สนุกมาก"))  # ควรได้ 'experience'
print(predict_category("มีใครรู้วิธีทำขนมครกไหม?"))  # ควรได้ 'question'


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1084/1084 ━━━━━━━━━━━━━━━━━━━━ 25s 20ms/step - accuracy: 0.8918 - loss: 0.3251 - val_accuracy: 0.9247 - val_loss: 0.2230
Epoch 2/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.9408 - loss: 0.1706 - val_accuracy: 0.9204 - val_loss: 0.2077
Epoch 3/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 40s 19ms/step - accuracy: 0.9665 - loss: 0.0969 - val_accuracy: 0.9194 - val_loss: 0.2417
Epoch 4/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.9840 - loss: 0.0601 - val_accuracy: 0.9086 - val_loss: 0.3159
Epoch 5/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.9896 - loss: 0.0356 - val_accuracy: 0.9043 - val_loss: 0.2998
Epoch 6/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.9970 - loss: 0.0107 - val_accuracy: 0.8903 - val_loss: 0.4699
Epoch 7/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.9989 - loss: 0.0070 - val_accuracy: 0.9097 - val_loss: 0.5033
Epoch 8/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 39s 18ms/step - accuracy: 0.9908 - loss: 0.03

In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from pythainlp.tokenize import word_tokenize

# โหลดข้อมูลจากไฟล์ CSV
data = pd.read_csv("Data_model.csv")

# แยกข้อความและป้ายกำกับ
texts = data["comments"].astype(str).tolist()
labels = data["label"].astype(str).tolist()

# Tokenization โดยใช้ PyThaiNLP
tokenized_texts = [word_tokenize(text, keep_whitespace=False) for text in texts]

# แปลงคำเป็นตัวเลข
max_words = 5000
max_len = 20
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(tokenized_texts)
sequences = tokenizer.texts_to_sequences(tokenized_texts)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# แปลง labels เป็นตัวเลข
unique_labels = list(set(labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}
y = np.array([label_map[label] for label in labels])

# แบ่งข้อมูลเป็น train/test (70/30) โดยใช้ random_state=42
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.3, random_state=42)

# สร้างโมเดล LSTM
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(len(unique_labels), activation='softmax')
])

# คอมไพล์โมเดล
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# ฝึกโมเดล
model.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test))

# ทำนายผลบนชุดทดสอบ
y_pred = np.argmax(model.predict(X_test), axis=1)

# คำนวณ Accuracy, Precision, Recall, และ F1-Score
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
class_report = classification_report(y_test, y_pred, target_names=unique_labels)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nClassification Report:\n", class_report)

# ฟังก์ชันสำหรับทำนายประเภทของข้อความ
def predict_category(text):
    tokenized = word_tokenize(text, keep_whitespace=False)
    sequence = tokenizer.texts_to_sequences([tokenized])
    padded = pad_sequences(sequence, maxlen=max_len, padding='post')
    pred = model.predict(padded)
    return unique_labels[np.argmax(pred)]

# ทดสอบการพยากรณ์
print(predict_category("เคยไปญี่ปุ่นมา สนุกมาก"))  # ควรได้ 'experience'
print(predict_category("มีใครรู้วิธีทำขนมครกไหม?"))  # ควรได้ 'question'


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1084/1084 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.8891 - loss: 0.3213 - val_accuracy: 0.9280 - val_loss: 0.2438
Epoch 2/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.9553 - loss: 0.1407 - val_accuracy: 0.9280 - val_loss: 0.2144
Epoch 3/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 41s 19ms/step - accuracy: 0.9765 - loss: 0.0806 - val_accuracy: 0.9237 - val_loss: 0.2785
Epoch 4/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.9936 - loss: 0.0230 - val_accuracy: 0.9054 - val_loss: 0.3515
Epoch 5/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step - accuracy: 0.9965 - loss: 0.0135 - val_accuracy: 0.9247 - val_loss: 0.3149
Epoch 6/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.9956 - loss: 0.0170 - val_accuracy: 0.9140 - val_loss: 0.3723
Epoch 7/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.9984 - loss: 0.0109 - val_accuracy: 0.9118 - val_loss: 0.4420
Epoch 8/10
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 42s 20ms/step - accuracy: 0.9986 - loss: 0.00

In [10]:
# ทดสอบการพยากรณ์
predict_category("เคยไปญี่ปุ่นมา สนุกมาก")  # ควรได้ 'experience'
# print(predict_category("มีใครรู้วิธีทำขนมครกไหม?"))  # ควรได้ 'question'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


'no_info'